## Import required libraries:

In [33]:
import pandas as pd

## Load the excel data:

In [71]:
ta = pd.read_excel('data/meta/Transit_Agencies_for_Visualization.xlsx', 
                   sheet_name='TC AgencyList')
ntd22 = pd.read_excel('data/ntd/TS2.2TimeSeriesSysWideOpexpSvc_2.xlsx', 
                      sheet_name=['UPT', 'VRM', 'FARES', 'VRH'])
ntd21 = pd.read_excel('data/ntd/TS2.1TimeSeriesOpExpSvcModeTOS_3.xlsx', 
                      sheet_name='UPT')

## Filter data:

In [72]:
# Remove missing NTD ID's
ta = ta.dropna(how='all')
ntd21 = ntd21.dropna(subset=['NTD ID'])

total = ntd22['UPT'].dropna(subset=['NTD ID'])
vrm = ntd22['VRM'].dropna(subset=['NTD ID'])
fares = ntd22['FARES'].dropna(subset=['NTD ID'])
vrh = ntd22['VRH'].dropna(subset=['NTD ID'])

def filterByMode(df, modes):
    return df[df['Mode'].isin(modes)]

# Filter bus data by required modes
bus_modes = ['MB', 'RB', 'CB', 'TB']
bus = filterByMode(ntd21, bus_modes)

rail_modes = ['CC', 'CR', 'HR', 'LR', 'MG', 'SR', 'YR']
rail = filterByMode(ntd21, rail_modes)

## Clean the data:

In [73]:
# Combine project ID's
ta['Project ID'] = ta['Project ID'].combine_first(ta['"Other" primary Project ID']).astype('int32')

# Drop unused columns
col22 = ['Last Report Year', 'Legacy NTD ID', 'Agency Name', 'Agency Status', 
         'Reporter Type', 'City', 'State', 'Census Year', 'Primary UZA Name', 
         'UZA', 'UZA Area SQ Miles', 'UZA Population', '2017 Status']
col21 = ['Last Report Year', 'Legacy NTD ID', 'Agency Name', 'Agency Status',
        'Reporter Type', 'City', 'State', 'Census Year', 'UZA Name', 'Mode', 'Service',
        'Mode Status', 'UZA', 'UZA Area SQ Miles', 'UZA Population', '2017 Status']

ta_clean = ta.drop(columns=['ShowIndividual', '"Other" primary Project ID', 'Primary UZA',
                            'UZA Name', 'Agency Name', 'Reporter Acronym'])

total_clean = total.drop(columns=col22)
vrm_clean = vrm.drop(columns=col22)
fares_clean = fares.drop(columns=col22)
vrh_clean = vrh.drop(columns=col22)

bus_clean = bus.drop(columns=col21)
rail_clean = rail.drop(columns=col21)

## Merge the data and calculate derived values:

In [79]:
def ntd_merge(df, name):    
    # Merge dataframes
    merge = pd.merge(ta_clean, df, how='left', on='NTD ID')
    group = merge.drop(columns=['NTD ID']).groupby('Project ID')
    
    # Sum, stack, and export to CSV
    stack = group.sum().stack()
    return stack.astype('int64').rename(name)
    
datasets = {
    'total': upt_clean,
    'bus': bus_clean,
    'rail': rail_clean,
    'vrm': vrm_clean,
    'total_fares': fares_clean,
    'vrh': vrh_clean
}

for name, df in datasets.iteritems():
    datasets[name] = ntd_merge(df, name)
    
# Average fares
datasets['fares'] = pd.Series(datasets['total_fares'] / datasets['total'], name='fares')
del datasets['total_fares']

# Average speed
datasets['speed'] = pd.Series(datasets['vrm'] / datasets['vrh'], name='speed')
del datasets['vrh']

## Export to CSV:

In [80]:
pd.concat(datasets.values(), axis=1).to_csv('data/output/ntd.csv', index_label=['id', 'year'])